In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [5]:
def max_returns_per_lookback(summary):
    # returns the chart of the max returns given summary.csv
    summary_max = summary.groupby('lookback')['avg_overall_return'].max()

    X = summary_max.index
    Y = summary_max

    plt.plot(X,Y)
    plt.show()
    
def generate_mktcap_size(mkt_cap_info):
    
    # adds new column 'Size' based on market cap
    
    # mkt cap definition, link: https://www.nasdaq.com/article/market-cap-explained-cm954078
    # Mega-cap: Market value of $300 billion or more;
    # Large-cap: Market value between $10 billion and $300 billion;
    # Mid-cap: Market value between $2 billion and $10 billion;
    # Small-cap: Market value between $250 million and $2 billion;
    # Micro-cap: Market value of less than $250 million.

    mkt_cap_info['Size'] = 'Mega'
    mkt_cap_info.loc[mkt_cap_info['MarketCap'].between(0, 250000000, inclusive=False), 'Size'] = 'Micro'
    mkt_cap_info.loc[mkt_cap_info['MarketCap'].between(250000000, 2000000000, inclusive=False), 'Size'] = 'Small'
    mkt_cap_info.loc[mkt_cap_info['MarketCap'].between(2000000000, 10000000000, inclusive=False), 'Size'] = 'Mid'
    mkt_cap_info.loc[mkt_cap_info['MarketCap'].between(10000000000, 300000000000, inclusive=False), 'Size'] = 'Large'

    return mkt_cap_info

    
def get_pair_type(pair_data, mkt_cap_info):
    # given pair data and separate mkt_cap_info, this will return the SIZE-SIZE pairing as a new column
    pair_type = []
    for i in range(0,len(pair_data)):
        pair = [str(mkt_cap_info.loc[mkt_cap_info['Symbol'] == pair_data.pair.str.split("-")[i][0], "Size"].values[0]), str(mkt_cap_info.loc[mkt_cap_info['Symbol'] == pair_data.pair.str.split("-")[i][1], "Size"].values[0])]
        pair.sort()
        pair_type.append('-'.join(pair))

    pair_data['pair_type'] = pd.Series(pair_type) 

    return pair_data.groupby(['pair_type']).mean().sort_values(by = 'sharpe_ratio', ascending = False)



In [50]:
summary = pd.read_csv(filepath_or_buffer = "~/Desktop/summary.csv")
# summary = original_file[['lookback', 'enter_threshold_size', 'exit_threshold_size', 'loss_limit', 'avg_sharpe_ratio', 'avg_overall_return', 'overall_return_std']]

idx_lookback = summary.groupby(['lookback'])['avg_overall_return'].transform(max) == summary['avg_overall_return']
idx_enter_threshold = summary.groupby(['enter_threshold_size'])['avg_overall_return'].transform(max) == summary['avg_overall_return'] 
idx_exit_threshold = summary.groupby(['exit_threshold_size'])['avg_overall_return'].transform(max) == summary['avg_overall_return'] 
idx_loss_limit = summary.groupby(['loss_limit'])['avg_overall_return'].transform(max) == summary['avg_overall_return'] 


summary['avg_overall_return'] = round(summary['avg_overall_return'],4)*100
best_params = pd.concat([summary[idx_lookback], summary[idx_enter_threshold], summary[idx_exit_threshold], summary[idx_loss_limit]], join = 'inner')

# get the pair size info
nyse_info = pd.read_csv(filepath_or_buffer = "~/Desktop/FYP/statistical-arbitrage-private-18-19/stock-names/nyse_tech_list.csv")
mkt_cap_info = generate_mktcap_size(nyse_info[['Symbol', 'MarketCap']])                        
                                           

/Users/brendantham/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/brendantham/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [55]:
# summary parameters file by max overall returns
best_params.sort_values(by = 'avg_sharpe_ratio', ascending = False)[['lookback', 'enter_threshold_size', 'exit_threshold_size', 'loss_limit', 'avg_sharpe_ratio', 'avg_overall_return', 'overall_return_std']].drop_duplicates()

,lookback,enter_threshold_size,exit_threshold_size,loss_limit,avg_sharpe_ratio,avg_overall_return,overall_return_std
82,25,2.0,0.50,-0.010,0.668280,24.93,0.453489
111,30,2.0,0.75,-0.005,0.462560,23.00,0.485430
28,15,2.0,0.50,-0.010,0.261036,16.95,0.432920
21,10,3.0,0.75,-0.005,0.249398,19.50,0.443372
26,10,3.0,1.00,-0.020,0.187420,15.96,0.450370
15,10,2.5,1.00,-0.005,0.105822,21.30,0.582905
54,20,2.0,0.50,-0.005,-0.029786,16.03,0.421483


In [57]:
best_params.sort_values(by = 'avg_sharpe_ratio', ascending = False)

,lookback,enter_threshold_size,exit_threshold_size,loss_limit,avg_sharpe_ratio,avg_overall_return,overall_return_std,uuid
82,25,2.0,0.50,-0.010,0.668280,24.93,0.453489,111434e3-d064-4989-ab85-d759b026671c
82,25,2.0,0.50,-0.010,0.668280,24.93,0.453489,111434e3-d064-4989-ab85-d759b026671c
82,25,2.0,0.50,-0.010,0.668280,24.93,0.453489,111434e3-d064-4989-ab85-d759b026671c
82,25,2.0,0.50,-0.010,0.668280,24.93,0.453489,111434e3-d064-4989-ab85-d759b026671c
111,30,2.0,0.75,-0.005,0.462560,23.00,0.485430,2c6e9a61-0707-4e64-9bd6-0e59f34ec2ee
111,30,2.0,0.75,-0.005,0.462560,23.00,0.485430,2c6e9a61-0707-4e64-9bd6-0e59f34ec2ee
111,30,2.0,0.75,-0.005,0.462560,23.00,0.485430,2c6e9a61-0707-4e64-9bd6-0e59f34ec2ee
28,15,2.0,0.50,-0.010,0.261036,16.95,0.432920,c8e80eda-b75e-46d6-9af9-2e5fea082f04
21,10,3.0,0.75,-0.005,0.249398,19.50,0.443372,a22c72b9-c6e0-45d2-b321-612408997b29
26,10,3.0,1.00,-0.020,0.187420,15.96,0.450370,1326137a-9a9a-4b63-9d3b-380818102f06


In [16]:
pair_data_good =  pd.read_csv(filepath_or_buffer = "~/Downloads/nyse-tech-daily/111434e3-d064-4989-ab85-d759b026671c.csv")
pair_data_good['returns_std_thousands'] = round(pair_data_good['returns_std']/1000,3)
get_pair_type(pair_data_good[['pair', 'sharpe_ratio', 'overall_return', 'returns_std_thousands']])


,sharpe_ratio,overall_return,returns_std_thousands
pair_type,,,
Large-Small,1.834386,0.187991,21.110333
Micro-Small,1.240984,0.431717,37.005500
Large-Mid,1.071801,0.225274,16.985813
Mid-Mid,0.643682,0.226737,15.375133
Small-Small,0.297450,0.152565,24.614333
Mid-Small,0.059445,0.503374,33.510200
Micro-Mid,-0.137801,-0.408978,36.046000
Large-Large,-1.686917,-0.465395,12.768000


In [20]:
pair_data_med =  pd.read_csv(filepath_or_buffer = "~/Downloads/nyse-tech-daily/c8e80eda-b75e-46d6-9af9-2e5fea082f04.csv")
pair_data_med['returns_std_thousands'] = round(pair_data_med['returns_std']/1000,3)
get_pair_type(pair_data_med[['pair', 'sharpe_ratio', 'overall_return', 'returns_std_thousands']])

,sharpe_ratio,overall_return,returns_std_thousands
pair_type,,,
Micro-Small,3.340162,0.604635,37.488500
Small-Small,1.115527,0.339393,29.907667
Large-Small,0.489912,0.299772,23.410333
Large-Mid,0.433038,0.186416,17.538875
Mid-Mid,0.053358,0.113611,15.673067
Micro-Mid,-0.059213,-0.184780,57.568000
Mid-Small,-0.089825,0.253314,26.907500
Large-Large,-1.840137,-0.680303,14.444000


In [21]:
pair_data_bad =  pd.read_csv(filepath_or_buffer = "~/Downloads/nyse-tech-daily/89651f4b-6c52-4c8c-8f68-8279c63de971.csv")
pair_data_bad['returns_std_thousands'] = round(pair_data_bad['returns_std']/1000,3)
get_pair_type(pair_data_bad[['pair', 'sharpe_ratio', 'overall_return', 'returns_std_thousands']])

,sharpe_ratio,overall_return,returns_std_thousands
pair_type,,,
Micro-Small,2.542863,0.506554,36.629000
Large-Small,0.645571,0.403501,24.034667
Small-Small,0.221369,0.015718,27.134000
Mid-Mid,0.121766,0.108520,14.773467
Mid-Small,0.034851,0.283354,29.210200
Micro-Mid,-0.265830,-0.289394,39.508000
Large-Mid,-0.738331,0.185205,17.088000
Large-Large,-1.574767,-0.452823,13.128000


In [60]:
pair_data_good.sort_values(by = "overall_return", ascending = False)

,pair,sharpe_ratio,overall_return,returns_std,returns_std_thousands
62,LXFT-NSP,0.690549,1.506739,67640.299696,67.640
19,EPAM-LXFT,0.728523,1.446005,60031.723921,60.032
12,HUBS-LXFT,0.638973,1.387230,75049.281682,75.049
4,NSP-STM,2.017444,1.067776,30168.690649,30.169
55,LXFT-MODN,1.596089,1.055199,21077.648227,21.078
5,AER-ELLI,1.252058,0.820399,27160.144729,27.160
63,ASGN-NOW,1.015616,0.774435,25536.205466,25.536
56,EPAM-PANW,16.926862,0.753443,22144.078603,22.144
47,FDS-MAN,1.313628,0.701424,14560.599446,14.561
42,LXFT-PANW,5.785681,0.689805,24615.012651,24.615


In [70]:
borrow_cost =  pd.read_csv(filepath_or_buffer = "~/Desktop/borrow_cost.txt", sep = '|')
borrow_cost

,,,,,,#BOF,2019.01.16,05:30:03
#SYM,CUR,NAME,CON,ISIN,REBATERATE,FEERATE,AVAILABLE,NaN
A,USD,AGILENT TECHNOLOGIES INC,1715006,XXXXXXXU1016,2.1400,0.2500,>10000000,NaN
AA,USD,ALCOA CORP,251962528,XXXXXXX21065,2.0512,0.3388,6300000,NaN
AAAU,USD,PERTH MINT PHYSICAL GOLD ETF,329705746,XXXXXXX61025,-9.9172,12.3072,2000,NaN
AABA,USD,ALTABA INC,278946664,XXXXXXX61017,2.1400,0.2500,>10000000,NaN
AABB,USD,ASIA BROADBAND INC,75216559,XXXXXXXL1008,2.1400,0.2500,100000,NaN
AABVF,USD,ABERDEEN INTERNATIONAL INC.,60152167,CA0030691012,-15.1820,17.5720,2000,NaN
AAC,USD,AAC HOLDINGS INC,169041192,XXXXXXX71083,1.2068,1.1832,100000,NaN
AACS,USD,AMERICAN COMMERCE SOLUTIONS,30207299,XXXXXXX91008,2.1400,0.2500,1300000,NaN
AACTF,USD,AURORA SOLAR TECHNOLOGIES IN,195675323,CA05207J1084,-33.3684,35.7584,5000,NaN
